In [2]:
import torch

In [3]:
torch.empty(5, 3)

tensor([[ 0.0000e+00,  2.0000e+00, -6.2170e+32],
        [ 1.5849e+29,  1.1210e-44,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00]])

In [4]:
torch.rand(5, 3)

tensor([[0.0290, 0.7536, 0.4018],
        [0.3132, 0.0028, 0.6553],
        [0.4663, 0.0589, 0.5638],
        [0.0405, 0.0353, 0.9120],
        [0.7697, 0.8639, 0.8471]])

In [13]:
x = torch.zeros(5, 3, dtype=torch.long)
x

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])

In [14]:
x = torch.tensor([3.3, 5])
x

tensor([3.3000, 5.0000])

In [15]:
x = x.new_ones(5, 3, dtype=torch.double)

In [16]:
torch.randn_like(x, dtype=torch.float)

tensor([[ 1.0136, -0.9861, -0.7379],
        [-0.6395, -0.2944,  1.5987],
        [-0.0534, -0.3049, -0.2917],
        [ 2.4367, -0.3919,  1.9873],
        [ 0.8687, -0.7861, -1.0693]])

In [17]:
y = torch.rand(5, 3)

In [18]:
x + y

tensor([[1.3015, 1.5363, 1.4194],
        [1.8083, 1.6153, 1.2227],
        [1.5909, 1.3526, 1.5095],
        [1.7160, 1.3498, 1.1375],
        [1.5191, 1.6559, 1.2612]], dtype=torch.float64)

In [21]:
x = torch.randn(4, 4)
y = x.view(16)
y

tensor([ 0.2453,  0.5872,  1.0513, -0.2208, -1.0339, -0.0752,  0.3489,  0.8131,
         0.8910,  0.7885, -0.5863,  1.5158, -0.3403,  0.4298, -0.2045,  1.6901])

## Autograd

In [180]:
x = torch.ones(2, 2, requires_grad=True)
x

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

In [181]:
y = x + 2
y

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)

In [182]:
y.grad_fn

In [199]:
z = y * y * 3
out = z.mean()
z, out

(tensor([[27., 27.],
         [27., 27.]], grad_fn=<MulBackward0>),
 tensor(27., grad_fn=<MeanBackward0>))

In [200]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


In [201]:
out.backward()
print(x.grad)

tensor([[18., 18.],
        [18., 18.]])


In [202]:
x = torch.randn(3, requires_grad=True)
y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([ -214.5830,   415.1213, -1153.9406], grad_fn=<MulBackward0>)


In [203]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


## Neural networks

### 1. Define the network

In [214]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        
        for s in size:
            num_features *= s
        
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [208]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [215]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0310, -0.0704,  0.1104,  0.0189,  0.1160,  0.0625, -0.1128, -0.0749,
         -0.0375, -0.1211]], grad_fn=<AddmmBackward>)


In [216]:
net.zero_grad()
out.backward(torch.randn(1, 10))